# Import model using pipeline component

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK.
- Import pipeline components from registry.
- Create `Pipeline` using registered components.
- Submit `Pipeline Job` in workspace
- Import model into workspace/registry from Huggingface/Azurestorage using above pipeline

**Motivations** - This notebook explains how to create model importing/publishing pipeline job in workspace using pipeline component registered in registry

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml import UserIdentityConfiguration

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace and the registry

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client_ws = MLClient.from_config(credential=credential)

ml_client_registry = MLClient(credential, registry_name="azureml")

# Retrieve an already attached MSI attached Azure Machine Learning Compute.
cluster_name = "cpu-standard-d13-msi"

# Note above compute has an MSI attached to it, which is necessary to get credentials and run cli/sdk v2 on that compute
# We also support OBO credentials (*in testing phase)

## 1.4 Ensure cluster has an MSI attached to it

In [ ]:
compute_cluster = ml_client_ws.compute.get(name=cluster_name)
if (
    compute_cluster.identity
    and compute_cluster.identity.type == "user_assigned"
    and compute_cluster.identity.user_assigned_identities[0].type == "managed_identity"
):
    print("Compute cluster has MSI attached to it")
else:
    raise (
        "Compute does not have an MSI attached to it, Kindly use the compute having an MSI attached"
    )

# 2. Load piepeline component from registry to create pipeline


## 2.1 Use registery handle to load the components in workspace

### Components
- import_model - Pipeline component which downloads the model, convert it into mlflow, validate locally and then register respectively

In [ ]:
import_model = ml_client_registry.components.get(
    name="import_model", version="0.0.8"
)

# 3. Create a pipeline job using pipeline component

## 3.1 Create pipeline object using necessary parameters

In [ ]:
pipeline_object = import_model(
                    model_id = "bert-base-uncased",
                    task_name = "fill-mask",
                    registry_name = "azureml-preview-test1"
                    )

## 3.2 Setting pipeline computes,identity 

In [ ]:
# Settings of compute and Identity
pipeline_object.settings.force_rerun = True
pipeline_object.settings.default_compute  = cluster_name
pipeline_object.identity = UserIdentityConfiguration()

# 4. Submit the import job


In [ ]:
# submit the pipeline job
pipeline_job = ml_client_ws.jobs.create_or_update(pipeline_object, experiment_name=f"{model_id} import")
# wait for the pipeline job to complete
ml_client_ws.jobs.stream(pipeline_job.name)